First things first.  Load the package.

In [33]:
Sys.setenv(R_CONFIG_ACTIVE = "jupyter")
suppressMessages({
  devtools::load_all()
  data(pcd)
})

metayer provides two methods for easing metaprogramming tasks, `wrapped_factory` and `with_monkey_patch`.

## wrapped_factory

It's easier to start with `wrapped_factory`.  This allows us to rewrite existing functions.  It replicates the function signature and provides a mechanism for calling the original function.  An example should be illustrative.

In [34]:
# a very simple function
orig <- function(x, y) x + y

A wrapper must have the signature `function(cmd, args, ...)` where `...` may be replaced with key value arguments to be passed in through the factory function.

In [35]:
# a simple wrapper
debug_wrapper <- function(cmd, args, label = NULL) { 
  
  # call wrapped_remap to create a list
  warg <- remap_symb(args) 

  # convert the remapped args into a string
  arg_str <- warg %>%
    purrr::imap_chr(
      \(k,v) sprintf("%s : %s", k, v)
    ) %>%
    paste0(collapse = "; ")

  # emit debugging information
  cat(
    glue(">>> called {label}: {arg_str}\n"), 
    file = stdout()
  )

  # call the original function
  do.call(cmd, args)
}

Now, produce a new function using the original function and the wrapper.

In [36]:
dbg_orig <- wrapped_factory("orig", debug_wrapper, label = "orig.debug")
dbg_orig(1, 2)

>>> called orig.debug: 1 : x; 2 : y

[1] 3

It's instructive to inspect the structure of `dbg_orig`.  It has the same function signature as the original function with the body of the wrapper.  Moreover, `cmd` has been replaced with `orig`, and `args` has been replaced with a symbol-mapped list.  This occurs in all places where `cmd` and `args` appear in the wrapper.

In [37]:
dbg_orig

function (x, y) 
{
    warg <- remap_symb(list(x = x, y = y))
    arg_str <- warg %>% purrr::imap_chr(function(k, v) sprintf("%s : %s", 
        k, v)) %>% paste0(collapse = "; ")
    cat(glue(">>> called {label}: {arg_str}\n"), file = stdout())
    do.call(orig, list(x = x, y = y))
}
<environment: 0x62ceab523868>

## with_monkey_patch

Do roughly the same thing as `wrapped_factory`, but with functions defined in a namespace and only temporarily.  Changes will be restored on exit.

The following is a simple example.  `time_caller` is a function that calls `Sys.Time`.  We'll assume that `time_caller` is defined in an external namespace, so we don't have access to `time_caller` directly.  Alternatively, `time_caller` could be used extensively elsewhere and  redefining it would be disruptive.  

In any case, it's default behavior is to call `Sys.Time` and return a `POSIXct` in the local time zone.

In [38]:
# suppose you couldn't change `date_caller` directly...
time_caller <- function() {
  Sys.time()
}
time_caller()

[1] "2024-09-23 20:09:47 PDT"

Our scenario is that we need `time_caller` to return something else.  In particular, we'd like the timezone to be GMT.  We achieve this by temporarily changing the behavior of `Sys.time` which `time_caller` invokes.

In [39]:
# apply a monkey patch that modifies the value returned
with_monkey_patch(
  "base::Sys.time",
  # adapt time_caller so that it returns a result in GMT 
  wrapper = function(cmd, args, func) {    
    t <- do.call(func, args)
    .POSIXct(t, "GMT")
  },
  {
    # could be nested deep down in the call stack...
    foo <- function() time_caller()
    foo()
  }
)

[1] "2024-09-24 03:09:48 GMT"

And check that everything is back to normal after our monkey patch call:

In [40]:
time_caller()

[1] "2024-09-23 20:09:49 PDT"

### a real example

While the above example is perhaps a bit contrived, metayer found this machinery useful for affixing knitr hooks to rmarkdown documents.  Specifically, deep in the call stack, `pkgdown::build_article` invokes `rmarkdown::html_document`.  We wanted to modify the output of the nested `html_document` call.  Rather than return the document that was created by default, we wanted to return a modified doc; in particular, one that set a particular knitr hook, `doc$knitr$knitr_hooks$metayer_hook = knitr_metayer_hook`.  

This approach has since been refactored away, but it remains a viable solution to the original problem.